In [2]:
import os

In [3]:
%pwd

'c:\\Text-summarization-End-to-End-project\\notebooks'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Text-summarization-End-to-End-project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
   root_dir: Path
   source_URL: str
   local_data_file: Path
   unzip_dir: Path

In [8]:
from src.TextSummarization.constants import *
from src.TextSummarization.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
            ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 

        create_directories([self.config.artifacts_root])     


    def get_data_ingestion_config(self) ->  DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config
    

In [17]:
import os
import urllib.request as request
import zipfile
from src.TextSummarization.logging.Logging import logger
from src.TextSummarization.utils.common import get_size



In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded {filename} with following info: \n{headers}")

        else:
            logger.info(f"File {self.config.local_data_file} already exists and file size : {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)





In [20]:
try:
    config = ConfigurationManager()

    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    print(e)



[2024-05-01 22:23:23,621: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-01 22:23:23,625: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-01 22:23:23,627: INFO: common: created directory at: artifacts]
[2024-05-01 22:23:23,628: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-01 22:23:29,402: INFO: 1388886024: Downloaded artifacts/data_ingestion/data.zip with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C9CA:166676:101957:14A0A9:66327384
Accept-Ranges: bytes
Date: Wed, 01 May 2024 16:53:25 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1257-QPG
X-Cache: M